In [ ]:
# default_exp linesdk

In [ ]:
#hide
import pickle
accessKey = ''
keypath = os.path.join(os.path.expanduser('~'), '.lineAccessKey')
if accessKey :
  with open (keypath, 'wb') as f:
    pickle.dump({
        'accessKey': accessKey
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
accessKey = creden['accessKey']
# print(accessKey)

In [ ]:
#hide
import pickle
credLocation = '/Users/nic/.lineLambda'
user =''
pw = ''
if user and pw:
  with open (credLocation , 'wb') as f:
    pickle.dump({
        'user': user,
        'pw': pw
    }, f)
if os.path.exists(credLocation):
  with open(credLocation , 'rb') as f:
    creden = pickle.load(f)
    PW = creden['pw']
    USER = creden['user']

else:
  PW = None
  USER = None


# lineSdk

> interact with product apis

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from lambdasdk.lambdasdk import Lambda
from linebot.models import TextSendMessage
from linebot import LineBotApi
from json import JSONDecodeError
import bz2,  boto3, base64, logging, os

In [ ]:
#export
class FunctionNames:
  '''determine function and resources name based on branchName'''
  def __init__(self, branchName:str = 'dev-manual'):
    self.branchName = branchName
  dumpToS3 = lambda self: f'product-dump-s3-{self.branchName}'
  updateProduct = lambda self: f'product-update-{self.branchName}'
  updateS3 = lambda self: f'product-update-s3-{self.branchName}'
  singleQuery = lambda self: f'product-get-{self.branchName}'
  allQuery = lambda self: f'product-get-all-{self.branchName}'
  inputBucket = lambda self: f'input-product-bucket-{self.branchName}'
  inventoryBucket = lambda self: f'product-bucket-{self.branchName}'
  

In [ ]:
#export
class Line:
  '''
    the main class for interacting with product endpoint
    user/pw are optional
  '''
  def __init__(self, 
               accessKey:str = ''):
    self.line_bot_api = LineBotApi(accessKey)
    
  def send(self, message:str = '', roomId:str=''):
    self.line_bot_api.push_message(roomId, TextSendMessage(text = message))
    return True
  
  @staticmethod
  def lambdaSend(event, _):
    line = Line(accessKey = os.environ.get('LINEACCESSKEY') or event['accessKey'])
    line.send(message = event['message'], roomId = event['roomId'])
    
class LineLambda:
  def __init__(self, user = None, pw=None, region = 'ap-southeast-1'):
    self.user = user
    self.pw = pw
    self.region = region
    
  def send(self, message:str = '', roomId:str='', 
           functionName = 'send-line',
           accessKey = ''
          ):
    lambda_ = Lambda(
      user = self.user,
      pw = self.pw,
      region = self.region
    )
    try:
      lambda_.invoke(
        functionName = functionName,
        input = {
          'message': message,
          'roomId': roomId,
          'accessKey': accessKey
        },
        invocationType = 'Event'
      )
    except JSONDecodeError as e:
      logging.exception('maybe there is no response')
    return True

# Testing

## generate dummy data for testing

In [ ]:
#hide
sampleInput = {
  "message": "helloWorld",
  "roomId": 'C9ba1d024ed36979222a2a2a8f67cfc9a' ,
  "accessKey": accessKey
}


## Create main class object

In [ ]:
line = Line(accessKey = accessKey)

## sendMessage

In [ ]:
%%time
result = line.send(roomId = sampleInput['roomId'], message= sampleInput['message'])
print(result)

LineBotApiError: LineBotApiError: status_code=429, request_id=609465ac-f9f4-479a-813f-9cbf80bfd9e9, error_response={"details": [], "message": "You have reached your monthly limit."}, headers={'Server': 'openresty', 'Content-Type': 'application/json', 'x-line-request-id': '609465ac-f9f4-479a-813f-9cbf80bfd9e9', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'x-frame-options': 'DENY', 'Content-Length': '50', 'Expires': 'Wed, 14 Oct 2020 01:38:06 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Wed, 14 Oct 2020 01:38:06 GMT', 'Connection': 'close'}

## Lambda

In [ ]:
%%time
result = Line.lambdaSend(sampleInput, '')

# Call function using deployed Lambda function

In [ ]:
%%time
lineLambda = LineLambda(USER, PW)
lineLambda.send(message='hello', roomId = sampleInput['roomId'])